<a href="https://colab.research.google.com/github/KimJisanER/code_jjambbong/blob/main/molecular_inputs_1204_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install -q condacolab
# import condacolab
# condacolab.install()

# import sys
# sys.path

# import condacolab
# condacolab.check()

# !mamba install -c conda-forge rdkit

In [ ]:
!pip install rdkit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 50.1 MB/s eta 0:00:00


In [ ]:
! pip install dgllife

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.1/226.1 kB 5.0 MB/s eta 0:00:00


In [ ]:
! pip install dgl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 51.6 MB/s eta 0:00:00


In [ ]:
!pip install -q transformers

In [ ]:
import numpy as np
import os
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
from rdkit import Chem
from rdkit.Chem import PandasTools
from rdkit.Chem import AllChem
import rdkit
import rdkit.Chem as Chem
from rdkit.Chem.Draw import IPythonConsole
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
import dgllife
from dgllife import data
import dgl
from tqdm import tqdm

DGL backend not selected or invalid.  Assuming PyTorch for now.


Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)


In [ ]:
from dgllife.data import MoleculeCSVDataset
from dgllife.utils import Meter, smiles_to_bigraph, CanonicalAtomFeaturizer, EarlyStopping, CanonicalBondFeaturizer, BaseAtomFeaturizer, ConcatFeaturizer
from dgllife.utils import BaseAtomFeaturizer, atom_type_one_hot, atom_degree_one_hot, \
    atom_formal_charge, atom_num_radical_electrons, \
    atom_hybridization_one_hot, atom_total_num_H_one_hot, atom_implicit_valence_one_hot, atom_is_aromatic, atom_is_chiral_center, atom_chirality_type_one_hot
from dgllife.utils.featurizers import BaseBondFeaturizer
from dgl.data.utils import save_graphs, load_graphs
from dgllife.model import AttentiveFPPredictor
from dgllife.model.gnn import AttentiveFPGNN
from dgllife.model.readout import AttentiveFPReadout
from functools import partial

In [ ]:
from transformers import TFBertModel, BertTokenizer,BertConfig
import re

tokenizer = BertTokenizer.from_pretrained("Rostlab/prot_bert", do_lower_case=False )
model = TFBertModel.from_pretrained("Rostlab/prot_bert", from_pt=True)

tokenizer_config.json:   0%|          | 0.00/86.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/81.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/361 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.68G [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint

In [ ]:
def load_sdf_file(file_path):
    suppl = Chem.SDMolSupplier(file_path)
    mols = [x for x in suppl if x is not None]
    df = PandasTools.LoadSDF(file_path)
    df['Molecule'] = mols
    df = df[['chembl_id', 'Molecule']]
    df = df.dropna()
    return df

def preprocess_data(df):
    # Molecular fingerprint 계산
    fp_list = []
    for mol in df['Molecule']:
        fp = AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits=2048, useChirality=True)
        fp_list.append(keras.utils.to_categorical(fp, num_classes=2))
        fp_list.append(fp)
    df['Fingerprint'] = fp_list

    # Fingerprint를 numpy 배열로 변환
    X = np.array(list(df['Fingerprint']))
    X = np.reshape(X, (X.shape[0], -1))

    return X

In [ ]:
sdf_file_path = '/content/drive/MyDrive/Class_A_0115.sdf'

In [ ]:
# SDF 파일 로드
df = load_sdf_file(sdf_file_path)

# 데이터 전처리
X = preprocess_data(df)

In [ ]:
df[['mw_freebase', 'alogp', 'pChEMBL_value_Ki', 'pChEMBL_value_Kd', 'pChEMBL_value_IC50', 'pChEMBL_value_EC50']] = df[['mw_freebase', 'alogp', 'pChEMBL_value_Ki', 'pChEMBL_value_Kd', 'pChEMBL_value_IC50', 'pChEMBL_value_EC50']].replace('None',None)

In [ ]:
df

,chembl_id,chembl_pref_name,target_chembl_id,target_accession,mw_freebase,alogp,bioactivities,pChEMBL_value_Ki,pChEMBL_value_Kd,pChEMBL_value_IC50,pChEMBL_value_EC50,ID,ROMol,Molecule,Fingerprint
0,CHEMBL100086,undefined,CHEMBL273,P19327,424.54,3.37,"assay_chembl_id: CHEMBL615874, assay_type: B, ...",None,None,8.66,None,,<rdkit.Chem.rdchem.Mol object at 0x79f13c27f840>,<rdkit.Chem.rdchem.Mol object at 0x79f1c51dd7e0>,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,CHEMBL100149,undefined,CHEMBL273,P19327,412.51,3.50,"assay_chembl_id: CHEMBL615874, assay_type: B, ...",None,None,7.57,None,,<rdkit.Chem.rdchem.Mol object at 0x79f13c27f8b0>,<rdkit.Chem.rdchem.Mol object at 0x79f1c51dd620>,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,CHEMBL100875,undefined,CHEMBL273,P19327,394.52,3.36,"assay_chembl_id: CHEMBL615874, assay_type: B, ...",None,None,7.70,None,,<rdkit.Chem.rdchem.Mol object at 0x79f13c27f920>,<rdkit.Chem.rdchem.Mol object at 0x79f1c51dd690>,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,CHEMBL100928,undefined,CHEMBL273,P19327,430.50,3.64,"assay_chembl_id: CHEMBL615874, assay_type: B, ...",None,None,7.47,None,,<rdkit.Chem.rdchem.Mol object at 0x79f13c27f990>,<rdkit.Chem.rdchem.Mol object at 0x79f1c51dd5b0>,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,CHEMBL101933,undefined,CHEMBL273,P19327,430.50,3.64,"assay_chembl_id: CHEMBL615874, assay_type: B, ...",None,None,7.68,None,,<rdkit.Chem.rdchem.Mol object at 0x79f13c27fa00>,<rdkit.Chem.rdchem.Mol object at 0x79f1c51dda10>,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188431,CHEMBL4299634,undefined,CHEMBL3562168,Q8BFU7,601.67,5.17,"assay_chembl_id: CHEMBL3816044, assay_type: B,...",None,None,None,6.54,,<rdkit.Chem.rdchem.Mol object at 0x79f0ff0ad9a0>,<rdkit.Chem.rdchem.Mol object at 0x79f13c27f610>,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ..."
188432,CHEMBL4299658,undefined,CHEMBL3562168,Q8BFU7,563.67,5.66,"assay_chembl_id: CHEMBL3816044, assay_type: B,...",None,None,None,6.35,,<rdkit.Chem.rdchem.Mol object at 0x79f0ff0ada10>,<rdkit.Chem.rdchem.Mol object at 0x79f13c27f680>,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
188433,CHEMBL4299669,undefined,CHEMBL3562168,Q8BFU7,561.65,5.44,"assay_chembl_id: CHEMBL3816044, assay_type: B,...",None,None,None,6.35,,<rdkit.Chem.rdchem.Mol object at 0x79f0ff0ada80>,<rdkit.Chem.rdchem.Mol object at 0x79f13c27f6f0>,"[0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
188434,CHEMBL1962870,undefined,CHEMBL3784908,Q5QD14,428.58,4.94,"assay_chembl_id: CHEMBL3788461, assay_type: F,...",None,None,5.32,None,,<rdkit.Chem.rdchem.Mol object at 0x79f0ff0adaf0>,<rdkit.Chem.rdchem.Mol object at 0x79f13c27f760>,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 188435 entries, 0 to 188435
Data columns (total 15 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   chembl_id           188435 non-null  object
 1   chembl_pref_name    188435 non-null  object
 2   target_chembl_id    188435 non-null  object
 3   target_accession    188435 non-null  object
 4   mw_freebase         188435 non-null  object
 5   alogp               188094 non-null  object
 6   bioactivities       188435 non-null  object
 7   pChEMBL_value_Ki    116547 non-null  object
 8   pChEMBL_value_Kd    3801 non-null    object
 9   pChEMBL_value_IC50  49281 non-null   object
 10  pChEMBL_value_EC50  28630 non-null   object
 11  ID                  188435 non-null  object
 12  ROMol               188435 non-null  object
 13  Molecule            188435 non-null  object
 14  Fingerprint         188435 non-null  object
dtypes: object(15)
memory usage: 23.0+ MB


In [ ]:
df_Ki = df.dropna(subset=['pChEMBL_value_Ki'])

In [ ]:
df_Ki=df_Ki[['chembl_id','pChEMBL_value_Ki', 'Fingerprint', 'target_accession']]

In [ ]:
df_Ki

,chembl_id,pChEMBL_value_Ki,Fingerprint,target_accession
5,CHEMBL104593,9.44,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...",P19327
6,CHEMBL105926,6.83,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",P19327
7,CHEMBL108026,7.46,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",P19327
9,CHEMBL1086958,8.21,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",P19327
10,CHEMBL1086959,6.85,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",P19327
...,...,...,...,...
187547,CHEMBL410333,6.82,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Q9Z2J6
187548,CHEMBL411315,7.48,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Q9Z2J6
187549,CHEMBL563949,7.55,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Q9Z2J6
188379,CHEMBL569766,8.15,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Q99527


In [ ]:

fasta_file = '/content/drive/MyDrive/class_A_bindingsite.fasta'

with open(fasta_file, 'r') as f:
    lines = f.readlines()

In [ ]:
seqs = []
headers = []
header_positions = []  # Store the positions of headers satisfying the condition
for i, line in enumerate(lines):
    if line.startswith('>'):
        headers.append(line.strip().upper())
        seqs.append('')
        if '>SP' in line and 'HUMAN' in line and 'HTR' in line:
            header_positions.append(i)  # Store the position of the header
    else:
        seqs[-1] += line.strip()

df_2 = pd.DataFrame(columns=['accession', 'sequence'])
data = []

for header, seq in zip(headers, seqs):
    accession = header.split('|')[1]
    data.append([accession, seq])

df_2 = pd.DataFrame(data, columns=['accession', 'sequence'])

In [ ]:
df_2

,accession,sequence
0,A0A2K5XZ43,___TFSYQVITSLLTLIAVVAISLQNVANLITDMVSVLVLPMAALY...
1,A0A673UJF2,___TFSYQVVTSLLTLIAVVAISLQNVANLITDMVSVLVLPMAALY...
2,A6I5H2,___TFSYQVITSLLTLIAVVAISLQNVANLITDMVSVLVLPMAALY...
3,A0A3Q7QQR7,___TFSYQVITSLLTLIAVVAISLQNVANLITDMVSVLVLPMAALY...
4,A0A2Y9LBV4,___TFSYQVITSLLTLIAVVAISLQNVANLITDMVSVLVLPMAALY...
...,...,...
22051,Q96RI9,_TYSPGPRSILYVLFGAAAVAIQLHTPTNLIADLVGVTVMPFSTVR...
22052,G1SUU9,_TYSPGPRAMLYVLLGAAVVAVQLHTPTHLIADLVGVTVMPFSTVR...
22053,A0A6P6I081,_TYSPGPRAILYVLFGAAMVAIQLHTPTNLIADLVGVTVMPFSTVR...
22054,A0A8C6HEP6,_SYAPWPRAILYVLLGAAVVAIQLHTPTNLVADLVGVTVMPFSTVR...


In [ ]:
data = pd.merge(df_Ki, df_2, left_on = 'target_accession', right_on = 'accession' ).drop(columns='target_accession')

data.to_pickle("/content/drive/MyDrive/Class_A_df_0219")

In [ ]:
sequences_Example = [re.sub(r"[UZOB]", "X", sequence) for sequence in data['sequence']]
ids = tokenizer.batch_encode_plus(sequences_Example, add_special_tokens=True, padding=True, return_tensors="tf")
input_ids = ids['input_ids']
attention_mask = ids['attention_mask']


In [ ]:
len(ids)

3

In [ ]:
embedding = model(input_ids)[0]

ResourceExhaustedError: Exception encountered when calling layer 'embeddings' (type TFBertEmbeddings).

{{function_node __wrapped__ResourceGather_device_/job:localhost/replica:0/task:0/device:GPU:0}} OOM when allocating tensor with shape[114530,53,1024] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:ResourceGather] name: 

Call arguments received by layer 'embeddings' (type TFBertEmbeddings):
  • input_ids=tf.Tensor(shape=(114530, 53), dtype=int32)
  • position_ids=None
  • token_type_ids=tf.Tensor(shape=(114530, 53), dtype=int32)
  • inputs_embeds=None
  • past_key_values_length=0
  • training=False

In [ ]:
embedding

<tf.Tensor: shape=(203, 1024), dtype=float32, numpy=
array([[-0.3291971 , -0.27789345,  0.32599688, ..., -0.32573476,
        -0.33244106,  0.3269173 ],
       [-0.3291971 , -0.27789345,  0.32599688, ..., -0.32573476,
        -0.33244106,  0.3269173 ],
       [-0.3291971 , -0.27789345,  0.32599688, ..., -0.32573476,
        -0.33244106,  0.3269173 ],
       ...,
       [-0.3292138 , -0.27790853,  0.32601354, ..., -0.32575145,
        -0.33245793,  0.3269339 ],
       [-0.3291805 , -0.27787855,  0.32598037, ..., -0.32571828,
        -0.33242434,  0.32690078],
       [-0.3292175 , -0.2779119 ,  0.32601723, ..., -0.32575515,
        -0.33246166,  0.32693762]], dtype=float32)>

In [ ]:
data

,chembl_id,pChEMBL_value_Ki,Fingerprint,accession,sequence
0,CHEMBL104593,9.44,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...",P19327,___TFSYQVITSLLTLIAVVAISLQNVANLITDMVSVLVLPMAALY...
1,CHEMBL105926,6.83,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",P19327,___TFSYQVITSLLTLIAVVAISLQNVANLITDMVSVLVLPMAALY...
2,CHEMBL108026,7.46,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",P19327,___TFSYQVITSLLTLIAVVAISLQNVANLITDMVSVLVLPMAALY...
3,CHEMBL1086958,8.21,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",P19327,___TFSYQVITSLLTLIAVVAISLQNVANLITDMVSVLVLPMAALY...
4,CHEMBL1086959,6.85,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",P19327,___TFSYQVITSLLTLIAVVAISLQNVANLITDMVSVLVLPMAALY...
...,...,...,...,...,...
114525,CHEMBL67464,7.68,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",P47936,___SSGQQIAVALCLMGSAVIIRLRKPSYFIADLASV-IFACNFVI...
114526,CHEMBL68641,8.72,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, ...",P47936,___SSGQQIAVALCLMGSAVIIRLRKPSYFIADLASV-IFACNFVI...
114527,CHEMBL947,8.74,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",P47936,___SSGQQIAVALCLMGSAVIIRLRKPSYFIADLASV-IFACNFVI...
114528,CHEMBL569766,8.15,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Q99527,EHQYVIGLFLSCYTFLFGFIVNEKMTIPDYFADILVA-DSLIEVFN...


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 114530 entries, 0 to 114529
Data columns (total 5 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   chembl_id         114530 non-null  object
 1   pChEMBL_value_Ki  114530 non-null  object
 2   Fingerprint       114530 non-null  object
 3   accession         114530 non-null  object
 4   sequence          114530 non-null  object
dtypes: object(5)
memory usage: 5.2+ MB


In [ ]:
def convert_data(data):
  data_array = np.array([dt for dt in data])
  return data_array

def convert_fpdata(data):
  fpdata_array = np.array([keras.utils.to_categorical(fp, num_classes=2) for fp in data])
  return fpdata_array

In [ ]:
X = data.drop(columns='pChEMBL_value_Ki')
y = data['pChEMBL_value_Ki'].astype('float32')

In [ ]:
X

,chembl_id,Fingerprint,accession,sequence
0,CHEMBL104593,"[[1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0...",P19327,"[[[-0.16569346, -0.11287488, -0.11425824, -0.0..."
1,CHEMBL105926,"[[1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0...",P19327,"[[[-0.16569346, -0.11287488, -0.11425824, -0.0..."
2,CHEMBL108026,"[[1.0, 0.0], [0.0, 1.0], [1.0, 0.0], [1.0, 0.0...",P19327,"[[[-0.16569346, -0.11287488, -0.11425824, -0.0..."
3,CHEMBL1086958,"[[1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0...",P19327,"[[[-0.16569346, -0.11287488, -0.11425824, -0.0..."
4,CHEMBL1086959,"[[1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0...",P19327,"[[[-0.16569346, -0.11287488, -0.11425824, -0.0..."
...,...,...,...,...
114525,CHEMBL67464,"[[1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0...",P47936,"[[[0.11569592, -0.040833525, -0.17255, -0.1731..."
114526,CHEMBL68641,"[[1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0...",P47936,"[[[0.11569592, -0.040833525, -0.17255, -0.1731..."
114527,CHEMBL947,"[[1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0...",P47936,"[[[0.11569592, -0.040833525, -0.17255, -0.1731..."
114528,CHEMBL569766,"[[1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0...",Q99527,"[[[0.02943428, -0.040906146, -0.0019021592, -0..."


In [ ]:
y

0         9.44
1         6.83
2         7.46
3         8.21
4         6.85
          ... 
114525    7.68
114526    8.72
114527    8.74
114528    8.15
114529    7.70
Name: pChEMBL_value_Ki, Length: 114530, dtype: float32

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=42)

In [ ]:
X_train

,chembl_id,Fingerprint,accession,sequence
88604,CHEMBL364458,"[[1.0, 0.0], [0.0, 1.0], [1.0, 0.0], [1.0, 0.0...",P42866,"[[[0.05017758, -0.023367923, 0.009527391, -0.0..."
67506,CHEMBL405234,"[[1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0...",P29274,"[[[0.06738183, -0.019542763, 0.04171653, -0.05..."
41965,CHEMBL404834,"[[1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0...",Q9Y5N1,"[[[0.053979173, -0.038492594, -0.0052488814, -..."
37814,CHEMBL77395,"[[1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0...",P21917,"[[[0.043839898, -0.103161626, -0.17593518, -0...."
96797,CHEMBL371223,"[[1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0...",P30559,"[[[-0.023466535, -0.12587252, -0.1068417, 0.03..."
...,...,...,...,...
33502,CHEMBL210717,"[[1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0...",P35462,"[[[0.024272803, -0.041849438, -0.02007928, -0...."
25562,CHEMBL197214,"[[1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0...",P50130,"[[[0.049319077, -0.019556755, 0.0077487165, -0..."
105849,CHEMBL409332,"[[1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0...",Q9Y5Y4,"[[[0.10961395, -0.046421766, -0.15180944, -0.1..."
63340,CHEMBL3827331,"[[1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0...",P30542,"[[[0.115763724, -0.053091355, -0.079522125, -0..."


In [ ]:
X_test

,chembl_id,Fingerprint,accession,sequence
79696,CHEMBL427683,"[[1.0, 0.0], [0.0, 1.0], [1.0, 0.0], [1.0, 0.0...",P41143,"[[[0.0600012, 0.00481162, 0.02759134, -0.07189..."
95715,CHEMBL3416879,"[[1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0...",Q62463,"[[[0.05806016, -0.06622588, -0.056183953, 0.00..."
64572,CHEMBL7909,"[[1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0...",P30542,"[[[0.115763724, -0.053091355, -0.079522125, -0..."
45709,CHEMBL152159,"[[1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0...",P43140,"[[[0.047590528, -0.06596921, -0.006795287, -0...."
61133,CHEMBL122622,"[[1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0...",P30542,"[[[0.115763724, -0.053091355, -0.079522125, -0..."
...,...,...,...,...
57008,CHEMBL452861,"[[1.0, 0.0], [0.0, 1.0], [1.0, 0.0], [1.0, 0.0...",P07550,"[[[0.03807997, -0.087925196, -0.031600744, -0...."
1414,CHEMBL477427,"[[1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0...",P19327,"[[[-0.16569346, -0.11287488, -0.11425824, -0.0..."
51891,CHEMBL193999,"[[1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0...",P23944,"[[[0.047590528, -0.06596921, -0.006795287, -0...."
83423,CHEMBL610032,"[[1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0...",P41145,"[[[0.052654184, -0.022806626, 0.009854864, -0...."


In [ ]:
X_val

,chembl_id,Fingerprint,accession,sequence
17639,CHEMBL479375,"[[1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0...",P50406,"[[[0.051022142, -0.012102532, -0.16300048, -0...."
14807,CHEMBL1258223,"[[1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0...",P47898,"[[[0.064125456, -0.13430855, -0.15913871, -0.1..."
104842,CHEMBL483991,"[[1.0, 0.0], [0.0, 1.0], [1.0, 0.0], [1.0, 0.0...",Q13258,"[[[0.10192498, -0.018801022, -0.14690912, -0.2..."
28828,CHEMBL362285,"[[1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0...",P14416,"[[[0.03286317, -0.049950566, -0.023394067, -0...."
56366,CHEMBL1824264,"[[1.0, 0.0], [0.0, 1.0], [1.0, 0.0], [1.0, 0.0...",P08588,"[[[0.09499164, -0.009476641, 0.058220673, -0.0..."
...,...,...,...,...
86957,CHEMBL3911926,"[[1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0...",P35372,"[[[0.05017758, -0.023367923, 0.009527391, -0.0..."
87842,CHEMBL47544,"[[1.0, 0.0], [0.0, 1.0], [0.0, 1.0], [1.0, 0.0...",P35372,"[[[0.05017758, -0.023367923, 0.009527391, -0.0..."
92650,CHEMBL3663528,"[[1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0...",O43613,"[[[0.055396277, -0.03240227, 0.0049790647, -0...."
25218,CHEMBL345357,"[[1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0...",Q95136,"[[[0.049319077, -0.019556755, 0.0077487165, -0..."


In [ ]:
train_inputs = [np.array(X_train['sequence']), np.array(convert_data(X_train['Fingerprint']))]
val_inputs = [np.array(X_val['sequence']), np.array(convert_data(X_val['Fingerprint']))]
test_inputs = [np.array(X_test['sequence']), np.array(convert_data(X_test['Fingerprint']))]

In [ ]:
# train_inputs = [np.array(convert_data(X_train['sequence'])), np.array(convert_data(X_train['Fingerprint']))]
# val_inputs = [np.array(convert_data(X_val['sequence'])), np.array(convert_data(X_val['Fingerprint']))]
# test_inputs = [np.array(convert_data(X_test['sequence'])), np.array(convert_data(X_test['Fingerprint']))]

TypeError: convert_data() missing 1 required positional argument: 'max_length'

In [ ]:
callback = [tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=20),
            tf.keras.callbacks.ModelCheckpoint(filepath='/content/drive/MyDrive/5_HTR/Class_A_model_{epoch:02d}-{val_loss:.3f}.h5',save_best_only=True)]

lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.02,
    decay_steps=100000,
    decay_rate=0.96)
optimizer_decay = keras.optimizers.Adam(learning_rate=lr_schedule)

In [ ]:
# from tensorflow import keras
# from tensorflow.keras import layers

# # 이미지 입력 경로
# image_input_shape = (24, 57)  # (높이, 너비)
# image_input = keras.Input(shape=image_input_shape)
# x = layers.Flatten()(image_input)  # 이미지를 24차원의 피처로 펼침

# # 피처 처리 경로
# x = layers.Dense(256, activation='relu')(x)
# x = layers.BatchNormalization()(x)
# x = layers.Dropout(0.2)(x)
# x = layers.Dense(128, activation='relu')(x)
# x = layers.BatchNormalization()(x)

# # 비트 입력 경로
# bit_input_shape = (2048, 2)  # (비트 입력 크기,)
# bit_input = keras.Input(shape=bit_input_shape)
# y = layers.Flatten()(bit_input)

# # 경로 병합
# merged = layers.concatenate([x, y])
# merged = layers.Dense(256, activation='relu')(merged)
# merged = layers.BatchNormalization()(merged)
# merged = layers.Dropout(0.2)(merged)
# merged = layers.Dense(128, activation='relu')(merged)
# merged = layers.BatchNormalization()(merged)

# # 회귀 모델 출력 레이어 추가
# outputs = layers.Dense(1)(merged)

# # 모델 생성
# model = keras.Model(inputs=[image_input, bit_input], outputs=outputs)

# # 모델 컴파일
# model.compile(optimizer='adam', loss='mse')

# # 모델 요약
# model.summary()


In [ ]:
image_input_shape = (31, 1024, 1)  # (높이, 너비, 채널)
bit_input_shape = (2048, 2)  # (비트 입력 크기,)

# 이미지 입력 경로
image_input = keras.Input(shape=image_input_shape)
x1 = layers.Dense(256, activation='relu')(image_input)
x1 = layers.BatchNormalization()(x1)
x1 = layers.Dropout(0.2)(x1)
x1 = layers.Dense(128, activation='relu')(x1)
x1 = layers.BatchNormalization()(x1)
x1 = layers.Flatten()(x1)

# 비트 입력 경로
bit_input = keras.Input(shape=bit_input_shape)
x2 = layers.Flatten()(bit_input)

# 경로 병합
merged = layers.concatenate([x1, x2])
merged = layers.Dense(512, activation='relu')(merged)
merged = layers.BatchNormalization()(merged)
merged = layers.Dropout(0.2)(merged)
merged = layers.Dense(256, activation='relu')(merged)
merged = layers.BatchNormalization()(merged)
merged = layers.Dropout(0.2)(merged)
merged = layers.Dense(128, activation='relu')(merged)
merged = layers.BatchNormalization()(merged)

# 회귀 모델 출력 레이어 추가
outputs = layers.Dense(1)(merged)

# 모델 생성
model = keras.Model(inputs=[image_input, bit_input], outputs=outputs)

# 모델 컴파일
model.compile(optimizer='adam', loss='mse')

# 모델 요약
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 31, 1024, 1)]        0         []                            
                                                                                                  
 dense (Dense)               (None, 31, 1024, 256)        512       ['input_1[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 31, 1024, 256)        1024      ['dense[0][0]']               
 Normalization)                                                                                   
                                                                                                  
 dropout (Dropout)           (None, 31, 1024, 256)        0         ['batch_normalization[0][0

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint

In [ ]:
checkpoint = ModelCheckpoint('/content/drive/MyDrive/HTR_only_model/best_model.h5', monitor='val_loss', save_best_only=True)

history = model.fit(train_inputs, y_train, validation_data=(val_inputs, y_val), epochs=100, callbacks=[checkpoint])

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type numpy.ndarray).

In [ ]:
from matplotlib import pyplot as plt
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model MSE')
plt.ylabel('mse')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='lower left',bbox_to_anchor=(1, 0))
plt.show()


In [ ]:
loss_value = model.evaluate(test_inputs, y_test)

In [ ]:
y_pred = model.predict(test_inputs)

In [ ]:
plt.plot(y_test, y_pred, "o", alpha=0.2)
plt.plot(range(0, 14), range(0, 14), 'r--')
plt.xlabel("Experimental pKi")
plt.ylabel("Predicted pKi")
plt.xlim(0, 13)
plt.ylim(0, 13)
plt.grid()

In [ ]:
from sklearn.metrics import r2_score
r2 = r2_score(y_test, y_pred)

In [ ]:
r2

In [ ]:
test_result = pd.concat([X_test.reset_index(drop=True), y_test.reset_index(drop=True), pd.Series(y_pred.squeeze())], axis=1)

In [ ]:
test_result.columns = ['chembl_id','Fingerprint','accession', 'sequence','pChEMBL_value_Ki', 'predicted_Ki']

In [ ]:
test_result

In [ ]:
test_result['abs_Error'] = abs(test_result['predicted_Ki'] - test_result['pChEMBL_value_Ki'])

In [ ]:
test_result

In [ ]:
test_result.sort_values('abs_Error', ascending=False)

In [ ]:
test_result.to_csv('/content/drive/MyDrive/5_HTR/all_aminergic_result_0906.csv')

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/5_HTR/all_aminergic_result_0906.csv')

In [ ]:
df = df[['chembl_id', 'accession', 'pChEMBL_value_Ki', 'predicted_Ki', 'abs_Error']]
df['predicted_Ki'] = df['predicted_Ki'].round(2)

In [ ]:
df

In [ ]:
df.sort_values('abs_Error', ascending=False)

In [ ]:
from_sdf = load_sdf_file(sdf_file_path)
mols = from_sdf[['chembl_id', 'Molecule']]

In [ ]:
mols

In [ ]:
for_result = pd.merge(df, mols, on='chembl_id').sort_values('abs_Error', ascending=False)
for_result = for_result.drop_duplicates(subset=['chembl_id', 'accession'])

In [ ]:
for_result

In [ ]:
error_top_50 = for_result.head(50)

In [ ]:
def visualize_molecule(mol):
    if mol is not None:
        PandasTools.RenderImagesInAllDataFrames(images=True)
        return mol

In [ ]:
visualize_molecule(error_top_50)

In [ ]:
images = []
for idx, row in error_top_50.iterrows():
    mol = row['Molecule']  # Mol 객체
    img = Draw.MolToImage(mol, size=(300, 300))
    images.append(img)

error_top_50['Molecule_Image'] = images

In [ ]:
error_top_50

In [ ]:
from rdkit import Chem
from rdkit.Chem import Draw
from PIL import Image, ImageDraw, ImageFont
import pandas as pd
import numpy as np

# 상위 50개 데이터 선택
top_50 = for_result.head(50)

# 분자 시각화 및 이미지 저장
images = []
labels = []
for idx, row in top_50.iterrows():
    mol = row['Molecule']  # Mol 객체
    img = Draw.MolToImage(mol, size=(300, 300))
    images.append(img)
    # chembl_id와 abs_Error를 라벨로 추가
    label = f"ChEMBL ID: {row['chembl_id']}\nabs_Error: {row['abs_Error']:.2f}"
    labels.append(label)

# 이미지를 하나의 이미지로 병합
num_rows = 10
num_cols = 5
merged_width = 300 * num_cols
merged_height = 300 * num_rows
merged_image = Image.new('RGB', (merged_width, merged_height))

for i, (img, label) in enumerate(zip(images, labels)):
    x_offset = (i % num_cols) * 300
    y_offset = (i // num_cols) * 300
    merged_image.paste(img, (x_offset, y_offset))

    # 라벨 추가
    draw = ImageDraw.Draw(merged_image)
    font = ImageFont.load_default()
    draw.text((x_offset, y_offset + 250), label, fill=(255, 255, 255), font=font)

# 이미지 저장
merged_image.save('molecule_visualization.png')

# 이미지 표시 (Jupyter Notebook 등에서 사용할 때)
merged_image.show()


In [ ]:
merged_image.show()

In [ ]:
# import numpy as np
# import matplotlib.pyplot as plt

# # 'error' 열의 분포를 히스토그램으로 시각화합니다.
# plt.hist(df['Error'], bins=200, alpha=0.5)

# # x축의 범위를 양쪽이 대칭이 되도록 설정합니다.
# x_range = np.abs(plt.xlim())
# plt.xlim(-max(x_range), max(x_range))

# # x축 0인 부분에 세로 점선을 그립니다.
# plt.axvline(x=0, color='orange', linestyle='dashed', alpha=0.3)

# # 90% 범위 계산
# lower_bound = np.percentile(df['Error'], 5)
# upper_bound = np.percentile(df['Error'], 95)

# # 90% 범위 경계에 세로 점선 그리기
# plt.axvline(x=lower_bound, color='black', linestyle='dashed', alpha=0.1)
# plt.axvline(x=upper_bound, color='black', linestyle='dashed', alpha=0.1)

# plt.title('Distribution of Error')
# plt.xlabel('Error')
# plt.ylabel('Count')

# # x축 눈금 설정
# plt.xticks(np.arange(np.floor(np.min(df['Error'])), np.ceil(np.max(df['Error'])) + 1, 1))

# # 그래프를 화면에 표시합니다.
# plt.show()


In [ ]:
df[df['chembl_id'] == 'CHEMBL39']

In [ ]:
df[df['chembl_id'] == 'CHEMBL4093971']

In [ ]:
df[df['chembl_id'] == 'CHEMBL3290004']

In [ ]:
df['chembl_id'].value_counts()

In [ ]:
unique_values = df['chembl_id'].value_counts()
one_occurrence_values = unique_values[unique_values == 1].index.tolist()

only_df = df[df['chembl_id'].isin(one_occurrence_values)]
not_only_df = df[~df['chembl_id'].isin(one_occurrence_values)]

In [ ]:
(only_df['abs_Error'].sum())/len(only_df)

In [ ]:
(not_only_df['abs_Error'].sum())/len(not_only_df)

In [ ]:
unique_values = df['chembl_id'].value_counts()
nine_occurrence_values = unique_values[unique_values == 9].index.tolist()

nn_df = df[df['chembl_id'].isin(nine_occurrence_values)]

In [ ]:

plt.plot(nn_df['pChEMBL_value_Ki'], nn_df['predicted_Ki'], "o", alpha=0.2)
plt.plot(range(0, 14), range(0, 14), 'r--')
plt.xlabel("Experimental pKi")
plt.ylabel("Predicted pKi")
plt.xlim(0, 13)
plt.ylim(0, 13)
plt.grid()

In [ ]:
unique_values = df['chembl_id'].value_counts()
tw_occurrence_values = unique_values[unique_values == 2].index.tolist()

tw_df = df[df['chembl_id'].isin(tw_occurrence_values)]

In [ ]:
plt.plot(tw_df['pChEMBL_value_Ki'], tw_df['predicted_Ki'], "o", alpha=0.2)
plt.plot(range(0, 14), range(0, 14), 'r--')
plt.xlabel("Experimental pKi")
plt.ylabel("Predicted pKi")
plt.xlim(0, 13)
plt.ylim(0, 13)
plt.grid()